# Pytorch fundamental

In [34]:
import torch
import numpy as np
torch.__version__

'1.10.0'

## Tensor object

In [4]:
scalar = torch.tensor(7)
print(scalar.ndim)
scalar.item()

0


7

In [8]:
vector = torch.tensor([7, 7])
print(vector.ndim)
vector.shape

1


torch.Size([2])

In [11]:
MATRIX = torch.tensor([[7, 8], 
                       [9, 10]])
print(MATRIX)
print(MATRIX.ndim)
print(MATRIX.shape)

tensor([[ 7,  8],
        [ 9, 10]])
2
torch.Size([2, 2])


In [12]:
# Tensor
TENSOR = torch.tensor([[[1, 2, 3],
                        [3, 6, 9],
                        [2, 4, 5]]])
TENSOR

tensor([[[1, 2, 3],
         [3, 6, 9],
         [2, 4, 5]]])

In [15]:
print(TENSOR.ndim)
print(TENSOR.shape)

3
torch.Size([1, 3, 3])


In [16]:
# Create a random tensor of size (224, 224, 3)
random_image_size_tensor = torch.rand(size=(224, 224, 3))
# Create a tensor of all zeros
zeros = torch.zeros(size=(3, 4))
ones = torch.ones(size=(3, 4))
# Use torch.arange(), torch.range() is deprecated 
zero_to_ten_deprecated = torch.range(0, 10) # Note: this may return an error in the future
# Create a range of values 0 to 10
zero_to_ten = torch.arange(start=0, end=10, step=1)

<ipython-input-16-25ef2e1c525d>:7: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  zero_to_ten_deprecated = torch.range(0, 10) # Note: this may return an error in the future


## Manupulating tensor

In [17]:
tensor = torch.tensor([1, 2, 3])
tensor + 10

tensor([11, 12, 13])

In [18]:
tensor * 10

tensor([10, 20, 30])

In [20]:
# torch function
torch.multiply(tensor, 10)

tensor([10, 20, 30])

In [21]:
# Element-wise multiplication (each element multiplies its equivalent, index 0->0, 1->1, 2->2)
print(tensor, "*", tensor)
print("Equals:", tensor * tensor)

tensor([1, 2, 3]) * tensor([1, 2, 3])
Equals: tensor([1, 4, 9])


In [22]:
# Matrix multiplication
torch.matmul(tensor, tensor)

tensor(14)

In [23]:
tensor_A = torch.tensor([[1, 2],
                         [3, 4],
                         [5, 6]], dtype=torch.float32)

tensor_B = torch.tensor([[7, 10],
                         [8, 11], 
                         [9, 12]], dtype=torch.float32)
# Transpose tensor_B
print(tensor_B.T)

tensor([[ 7.,  8.,  9.],
        [10., 11., 12.]])


## Finding the min, max, mean, sum, etc (aggregation)

In [24]:
x = torch.arange(0, 100, 10)

In [32]:
print(x.min())
print(x.max())
print(x.type(torch.float32).mean())
print(x)

tensor(0)
tensor(90)
tensor(45.)
tensor([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90])


In [33]:
torch.max(x), torch.min(x), torch.mean(x.type(torch.float32)), torch.sum(x)

(tensor(90), tensor(0), tensor(45.), tensor(450))

In [35]:
# numpy to tensor
array = np.arange(1.0, 8.0)
tensor = torch.from_numpy(array)
array, tensor

(array([1., 2., 3., 4., 5., 6., 7.]),
 tensor([1., 2., 3., 4., 5., 6., 7.], dtype=torch.float64))

In [36]:
# tensor to numpy
tensor = torch.ones(7) # create a tensor of ones with dtype=float32
numpy_tensor = tensor.numpy() # will be dtype=float32 unless changed